# 020 Reading and Writing Data Files

> COM6018

*Copyright © 2023–2025 Jon Barker, University of Sheffield. All rights reserved.*


## 1 Introduction

### 1.1 Storing data in files

In this course, we will be dealing with many different sets of data. Almost all of these will start life as a file stored on a computer, which we will need to read into our programs. The programs that we write will be transforming or analysing these data in some way. The results will then typically need to be written back to a file so that they can be used by other programs or people. So, understanding how to read and write data files is a key skill for a data scientist.

When storing data in a file, we need to use a standard format so that the data can be easily read by others. There are a huge variety of different data file formats that are used in data science; some are very general purpose, others are specialised `domain-specific' formats. In these notes, we will be introducing some of the most common formats that you are likely to encounter.

### 1.2 Human-readable vs. Machine-readable

One of the first considerations when writing data to a file is whether or not it needs to be **human-readable**. By human-readable, we mean that the data can be directly inspected and understood by a human, i.e., typically by being represented using ASCII characters. In many cases, this is not required and we can use a **machine-readable** format (also known as a **binary format**). A binary format will typically be more compact (i.e., it will consume less storage space) and faster to read and write. However, for small datasets the storage and read/write times are not significant, and a human-readable format is often easier to work with. In this course, we will start with a relatively small dataset and use human-readable formats for all of our data files.

There are many different human-readable data formats, but two of the most common ones for data science are CSV and JSON. We will be looking at these formats in some detail in these notes.

### 1.3 Some terminology

Before describing the different data formats, we need to introduce some terminology. When we talk about a dataset, we typically refer to a collection of **data items** describing the objects in the set. Each data item is described by a collection of **fields**, where each field describes a different aspect of the object. For example, a dataset might contain information about a number of people. Each person would be a data item and the fields would be things like name, age, height, weight, etc. Note that the fields may be of different types; e.g., the name field would be a string, the age field would be an integer, the height field would be a float, etc.

In data science, and particularly in machine learning, we are often dealing with datasets that represent examples of some larger population. For example, we may have a collection of 10,000 images of cats from which we want to learn about all possible images of cats. In machine learning, each data item is more commonly called a **sample** and the fields are called **features**. So depending on the context (or who we are talking to!) we might talk about a dataset containing 10,000 samples each with 100 features; or a dataset having 10,000 data items each with 100 fields.

In these notes on file formats, we will use the terms 'data item' and 'field.' Later in the course, when discussing machine learning, we will more often use the terms 'sample' and 'feature.'

## 2 CSV Files

### 2.1 The CSV file format

CSV stands for 'Comma Separated Values'. It is a very simple format that is commonly used for tabulated data, i.e. data that might be stored in a spreadsheet. It is therefore ideal for our typical case, where we have a dataset containing a number of data items, each with a number of fields.

In CSV, each row stores a single data item and, within the row, the field values are listed separated by commas (hence the name CSV). Optionally, the first row of the file can contain the names of the fields.

For example, a CSV file of wind farm data might be storing a wind farm ID, the number of turbines, the turbine height, and the maximum kW power of each wind farm. This would be stored in a file called, for example, [data/020/windfarm.csv](data/020/windfarm.csv), with contents like this,

```csv
"id", "turbines", "height", "power"
"WF1355", 13, 53, 19500
"WF1364", 3, 60, 8250
"WF1356", 12, 60, 24000
"WF1357", 36, 60, 72000
```

The above example stores data for just four wind farms. In practice, a CSV file might contain thousands or hundreds of thousands of data items. Also, in a real dataset there may be a far greater number of fields, e.g., latitude, longitude, year of installation, turbine manufacturer, etc.

### 2.2 Reading a CSV file with `csv.reader`

For handling csv files in Python, we can use the `csv` module, which is part of the Standard Library. This has a method called `reader` which will take a file handle and return an **iterator object** that allows us to read each row of the file as a list of field values. In the example below, this has been used in a list comprehension to load the entire dataset as a list of lists.

In [ ]:
import csv
with open('data/020/windfarm.csv') as csvfile:
    windfarm_reader = csv.reader(csvfile, skipinitialspace=True)
    # Read each row (as a list) and store them as a list of lists.
    data = [row for row in windfarm_reader]

💡 Tip: In practice, it is good to be explicit about the file encoding, especially when working with real-world data that may include accented or non-ASCII characters. You can do this by passing the argument `encoding='utf-8'` to `open()`, e.g.
`with open('data/020/windfarm.csv', encoding='utf-8') as csvfile:`
Remember that the file path is interpreted relative to your current working directory, which may differ when running code from a notebook versus a terminal.

🔺 Note the use of the parameter setting `skipinitialspace=True`. This tells the reader to ignore **whitespace** (i.e., space and tab characters) that might occur after the comma separators in the file. This is important because if this is not done then spaces in the file would be incorporated into the field values (which is not normally what you want.)

We can now access the data, for example,

In [ ]:
# Print the entire dataset
print(data)
# Print the 2nd entry
print(data[1])
# Print the 3rd field of the 2nd entry
print(data[1][2])

This code has read the data, but the resulting data are not stored in a very convenient format. The data are returned as a list of lists. The first element of the list contains the names of the fields that are stored as a list of strings. Subsequent elements in the list contain the rows of the csv file with each of these rows stored as a simple list of field values. To interpret these lists, we have to remember what the order of the list means (or use the names in `data[0]`). A further issue is that all the field values are being treated as strings, e.g. the power for the first windfarm is 19500 kW, where 19500 is a numeric value but is stored as the character string '19500'. We would need to take extra steps to convert the fields into the correct types.

### 2.3 A better way - using `csv.DictReader`

Storing our dataset as a list of lists makes it hard to work with. A much more useful representation would be as a **list of dictionaries**. In this case,  each dictionary would store a single data item, and the dictionary entries would store the field values, indexed by their field names. For example, the data would look like this,

```json
[
    {
        "id": "WF1355",
        "turbines": 13,
        "height": 53,
        "power": 19500
    },
    {
        "id": "WF1364",
        "turbines": 3,
        "height": 60,
        "power": 8250
    },
    // etc
]
```

Although this representation has a lot of redundancy (i.e., the field names are repeated for each data item), it is much easier to work with, and the redundancy is not a problem for small datasets. (Later, we will be looking at a package called Pandas that provides much more efficient ways of dealing with very large datasets.)

Fortunately, the csv module allows us to read the file into this format very easily. We simply need to replace the `reader` object with a `dictReader`. This is done as follows.

In [ ]:
import csv
with open('data/020/windfarm.csv') as csvfile:
    windfarm_reader = csv.DictReader(csvfile, skipinitialspace=True)
    # ... windfarm_reader is now a DictReader which will read each row as a dictionary
    # The line below remains exactly the same but now generates a list of dicts.
    data = [row for row in windfarm_reader]

Note how the above code snippet compares with the example in the previous section. We have simply had to replace `csv.reader` with `csv.DictReader`.

We can now access the data using list indexing to retrieve data items and dictionary indexing to access specific fields. For example,

In [ ]:
# print the 1st entry
print(data[0])
# print the power output of the first entry
print(data[0]['power'])

This is both easier to write (as we do not have to remember the order of the fields in the original file) and much easier to read, i.e., `data[0]['power']` is much more meaningful than `data[0][3]`.

### 2.4 Dealing with numeric versus string fields

Note, we still have the problem in the example above that all fields are read in and stored as strings.

A solution to this is to use the `quoting` parameter of the `DictReader` to specify that unquoted values should be converted into floats; quoted values remain as strings. This is done as follows.

In [ ]:
import csv
with open('data/020/windfarm.csv') as csvfile:
    windfarm_reader = csv.DictReader(csvfile, skipinitialspace=True,  quoting=csv.QUOTE_NONNUMERIC)
    # ... windfarm_reader is now a DictReader which will read each row as a dictionary
    # The line below remains exactly the same but now generates a list of dicts.
    data = [row for row in windfarm_reader]

So we can now see that the numeric values are being stored as numbers and not strings.

In [ ]:
# print the 1st entry
print(data[0])

This is still not perfect because all numeric values have been stored as floats. We might have hoped that the csv library could infer whether values are integers or floats by the presence of a decimal point. Unfortunately, this is not the case.

We can, however, easily process the data after having read it to convert the values of specific fields to integers if we want to. For example, the following compact piece of code will cast the 'turbine' field (which stores the integer number of turbines in the farm) into an integer value. Note that it is effectively regenerating the list by processing each dictionary in turn. (Do not worry too much about the syntax of this code at this stage.)

In [ ]:
# Convert the power values to integers
data = [{**row, 'turbines': int(row['turbines'])} for row in data]

# Note how the turbines field is now an integer
print(data)

A more difficult issue is that we have relied on the csv file having quotes around all the string values. In fact, many csv files do not follow this convention, i.e. string-valued fields may appear unquoted. In this case the "QUOTE_NONNUMERIC" option would cause the parsing to fail as it would try to convert these string into numeric values.

If our csv file has unquoted strings, then the easiest solution is to go back to treating everything as a string value and then doing explicit type conversions ourselves using code like the example above.

Fortunately, in the weeks to come, when we look at the Pandas package, we will see more powerful csv readers that will handle a lot of these issues for us.

### 2.5 Writing CSV files

Just as we can read CSV data using `csv.DictReader`, we can also write data using `csv.DictWriter`. The DictWriter object takes a list of field names that define the column order and can write either individual rows or a complete list of dictionaries. For example:

In [ ]:
with open('data/020/windfarm_copy.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=data[0].keys())
    writer.writeheader()
    writer.writerows(data)

The `newline=''` argument prevents extra blank lines from appearing in the output on Windows systems. The resulting file can be opened directly in spreadsheet programs such as Excel or LibreOffice Calc.

## 3 The JSON data format

The CSV files that we looked at in the last section are convenient when each field in our data entry can be represented by a simple type such as a string or a number. However, in many cases, the field values might themselves have more complex types such as lists or even dictionaries.

For example, imagine we have weather data for a large number of cities around the world for a given year. For each city (one data entry), we may have simple fields that store, for example, the name and the country, but the weather data might be recorded for each month, i.e., twelve values. Further, for each month we might have separate recordings for, say, the daily rainfall, minimum and maximum temperature, the number of snowy days, etc, which would be best stored as a dictionary. So, within the data entry, we have a list of twelve months, and each month is in turn stored as a dictionary. Our resulting data might look something like this,

```json
[
  {
    "city": "Amsterdam",
    "country": "Netherlands",
    "monthlyAvg": [
      {
        "high": 7,
        "low": 3,
        "snowDays": 4,
        "rainfall": 68
      },
      {
        "high": 6,
        "low": 3,
        "snowDays": 2,
        "rainfall": 47
      }
      // etc
    ]
  }
  // etc
]
```

In the data above, we still have a list of dictionaries, where each dictionary represents a single data entry (that is, one city). However, within a data entry we now have a field "monthlyAvg" which is itself a  list of dictionaries (i.e. weather data for each of the twelve months).

This kind of nested data structure is not easily represented in a CSV file. For this type of data, we need a more flexible format. One such format is **JSON** (**JavaScript Object Notation**). This is a widely used format for storing data and is used extensively in web applications. It is also a very convenient format for storing data in Python and is now widely used in data science.

JSON files are text files which are usually given the file extension '.json'. Their contents look like the text in the example above (in fact, the text above is a snippet of a larger JSON file with '//etc' added where text has been deleted). Dictionary entries are introduced with '{' and terminated with '}'. Lists are introduced with '[' and terminated with ']'. The values in the dictionary entries can be strings, numbers, lists or even other dictionaries. The values in the lists can be strings, numbers, lists or dictionaries. So, JSON files can be used to store arbitrarily complex data structures. Unlike CSV files, which can vary in their format (e.g., do strings have quotes or not? What character is used as the quote? etc.), the JSON format is strictly defined and so although it can store more complex data, there is no ambiguity in how the data are stored. This makes it easy to write tools for reading and writing JSON files, and there are many such tools available.

(For the full specification of the JSON format see <https://www.json.org/json-en.html>. Note that in the specification, the term "object" is used to refer to what we have been calling a dictionary, and the term "array" is used to refer to what we have been calling a list. This is largely due to JSON's origins in JavaScript.)

### 3.1 Reading JSON files

As with CSV files, Python has a Standard Library module for reading and writing JSON files. The module is called `json` and is imported with `import json`.

The `json` module has two main functions, `load` and `dump`. The `load` function is used to read a JSON file and convert it into a Python data structure. The `dump` function is used to convert a Python data structure into a JSON file.

In the example below, we use the `load` function to read the JSON file `data/020/climate.json` and convert it into a Python list, which we have called `climate_data`.

In [9]:
import json

with open('data/020/climate.json') as jsonfile:
    climate_data = json.load(jsonfile)

Notice how this code is even simpler than the CSV reading code. This is largely because the strictness of the JSON format means that the load function does not require any additional configuration.

We can now print out the first entry using the following.

In [ ]:
print(climate_data[0])

There are a few things to note when comparing this to the earlier CSV example.

- There is now no ambiguity about string vs numeric types because the JSON format strictly insists that all strings are quoted. In fact, it even insists that the double quote (\") symbol is used for quoting.
- Numeric values can have an integer or float type. In the example above, all values are integers, but if the values had contained a decimal point, then they would have been read as floats.
- The JSON reader converts JSON arrays into Python lists. There is no JSON equivalent of a tuple. i.e., in Python tuples are delimited by '(' and ')', but in JSON there is no such notation.
- The JSON standard requires that the object keys are strings. This is stricter than a general Python dictionary. Python is happy to have dictionaries that have keys of non-string types (e.g., ints). This means that not every Python dictionary can be converted into a JSON object. i.e., the JSON format is very flexible, but not as flexible as Python. In practice, this is rarely an issue for the types of data that are used in data science.

### 3.2 Writing JSON files

Writing JSON files is as easy as reading them. To write a JSON-compatible Python data structure to a file, we use the `dump` function. For example, to write out the climate data that we have just read in, we can use,

In [ ]:
with open('data/020/climate_copy.json', 'w') as jsonfile:
    json.dump(climate_data, jsonfile)

We can check the contents of this file,

In [ ]:
# %%bash
# head data/020/climate_copy.json

(Note, remove the `#` if you are running this in a terminal window. Lines have been commented out so that the long output does not appear in these notes.)

Notice that the whole file has appeared on one line. This is nice and compact, but not very human-readable! To make the dump function split the output over lines and indent the nested structure we simply need to pass one additional parameter, `indent=4` (where the value of the parameter specifies how many columns to use when indenting; 4 and 2 are commonly used values). For example,

In [ ]:
with open('data/020/climate_copy.json', 'w') as jsonfile:
    json.dump(climate_data, jsonfile, indent=4)

We can check the contents of this file by printing the first few lines in a terminal window, e.g., using linux,

In [ ]:
%%bash
head data/020/climate_copy.json

As we mentioned, not all Python data structures can be directly translated into valid JSON. For example, suppose that we had a list of dictionaries that had tuple entries. What would happen?

In [ ]:
import json
my_data = [{'name': 'Alice', 'favourite_numbers': (1, 2, 3)},
           {'name': 'Bob', 'favourite_numbers': (4, 5, 6)}]

with open('tmp/numbers.json', 'w') as jsonfile:
    json.dump(my_data, jsonfile)

with open('tmp/numbers.json') as jsonfile:
    my_data_read_back_in = json.load(jsonfile)

print('favourite_numbers starts as a:')
print(type(my_data[0]['favourite_numbers']))
print('favourite_numbers is read back as:')
print(type(my_data_read_back_in[0]['favourite_numbers']))

Note that the tuple has been stored as a JSON array, so on reading it is interpreted as a Python list, that is, the `my_data_read_back_in` variable is not the same as the original `my_data` variable. The values stored in both are the same, but different data types are used (i.e. a list versus a tuple). This can lead to subtle bugs if you are not careful.

In the more extreme example below, we will try writing a dictionary that uses an integer key.

In [ ]:
import json
my_data = [{1: 'chocolate', 2: 'vanilla'},
            {1: 'raspberry', 2: 'chocolate'}]

with open('tmp/icecream.json', 'w') as jsonfile:
    json.dump(my_data, jsonfile)

with open('tmp/icecream.json') as jsonfile:
    my_data_read_back_in = json.load(jsonfile)


print(my_data_read_back_in)

Note that in this case, the key values were converted to strings. Again, this might lead to problems if we later expected the key values to be numeric.

These issues do not usually cause real difficulties, but it is worth being aware of them.

### 3.3 JSON Lines format

One of the nice features of the CSV format is that each data entry is stored on a separate line. This makes it very easy to read items from large files one at a time, i.e., note how the csv.reader and csv.DictReader objects iterate over a file, line by line, returning a data entry each time.

With the standard JSON format this is not possible. JSON does not dictate how white space is used and as we have seen, it is typical to store a single data-entry over many lines using indentation to indicate the nested structure. Even if we insisted on storing each data-entry on a single line there is a further problem: the JSON format for a list of dictionaries requires the whole list to be enclosed in square brackets. This means that a JSON parser will be looking for the closing square bracket before it can return any data. This is not a problem for small files, but for large files it means that the whole file must be read into memory before any data can be returned.

To make JSON more friendly for storing large data science datasets, the `JSON Lines` format was developed (typically stored with the file extension `.jsonl`). In this format, we simply store each data entry as a valid JSON string **on a separate line** terminated by a newline character. Note that although each line is a valid JSON object (and can be parsed with json.loads), the file as a whole is not. i.e., the list of data entries is not enclosed in square brackets, and the data entries are not separated by commas. However, this file, like a CSV file, can be read and parsed line by line. This is particularly useful for processing large data science datasets that may be too large to fit into memory.

To make this clear, compare the JSON and JSON Lines formats for the ice cream flavour data that we had above.

- First, as it might appear in a JSON file...

```json
[
 {
    "name": "Alice",
    "favourite_numbers": [1, 2, 3]
 },
 {
    "name": "Bob",
    "favourite_numbers": [4, 5, 6]
 }
]
```

- ... and now as it would appear in a JSON Lines file.

```json
 {"name": "Alice", "favourite_numbers": [1, 2, 3]}
 {"name": "Bob", "favourite_numbers": [4, 5, 6]}
```

Note that we no longer enclose the items in a list and we no longer separate them with commas. However, we insist that each item has its own line.

The JSON Lines format is a very simple convention and the full specification can be found on this single page website, <https://jsonlines.org/>

#### 3.3.1 Reading JSON lines files

Note, because `.jsonl` is not a valid `.json` file, it cannot be read directly with the builtin `json.load` function. Also, there is no jsonl file reader built into the standard library. However, we can easily read `.jsonl` files with a few lines of Python. To do this, we can use the `json.loads` function, which will convert a single string into a JSON object. Using this, we can read the file line-by-line and convert each line as we go. For example,

In [ ]:
import json

with open('data/020/climate.jsonl') as jsonfile:
    climate_data = [json.loads(line) for line in jsonfile]

Note, the example above is not particularly useful as we have still read the entire data into memory. If the dataset were very large we would more typically be processing the data while we read, i.e., we might do something like the following.

In [ ]:
import json

with open('data/020/climate.jsonl') as jsonfile:
    for line in jsonfile:
        climate_data_entry = json.loads(line)
        # do something with this data entry

#### 3.3.2 Writing JSON lines files

Writing files in JSON Lines format is just as easy. We simply need to convert each data entry into a JSON string and write it to a file. The `json.dumps` function will convert the JSON object into a string for us. So, for example,

In [ ]:
import json

with open('data/020/climate.jsonl', 'w') as jsonfile:
    for data_entry in climate_data:
        jsonfile.write(json.dumps(data_entry) + '\n')

Note how we are adding a newline character ('\n') to the end of each string. This is important as it will ensure that each data entry is stored on a separate line.

In many real-world applications, JSON Lines files are not written all at once but are appended to over time. For example, a data-collection script might record one measurement or log entry at a time and add it as a new line to an existing .jsonl file.

To support this, you can open the file in append mode rather than write mode by using 'a' instead of 'w' in the `open()` call:

In [20]:
with open('data/020/climate.jsonl', 'a', encoding='utf-8') as jsonfile:
    for data_entry in climate_data:
        jsonfile.write(json.dumps(data_entry) + '\n')

Each new line you write is added to the end of the file without re-reading or rewriting earlier data. This line-by-line append behaviour is one of the main reasons JSON Lines is so popular for streaming, incremental, or continuously growing datasets, such as logs, sensor readings, or large-scale web data collections.

Note that because the JSON lines format squashes the entire object onto a single line, it is no longer quite as human-readable as the standard JSON format. However, for most popular coding editors (such as VSCode) there are now editor plugins that will allow you to browse JSON lines files in a readable way. The JSON lines format is therefore a great compromise between machine and human readability: It has the flexibility of JSON while allowing the line-by-line processing of CSV.

*Copyright © 2023–2025 Jon Barker, University of Sheffield. All rights reserved.*